Note to self: `jupyter nbconvert --to rst text/demo_notebook.ipynb`.

# Error handling

To report an error (which in Python, is called an `Exception`), just `raise` it:

In [ ]:
import math

def _sqrt(x: float, tol: float = 1e-5) -> float:
    """Computes the square root of a number `x` up to a given convergence `tol`, using the Newton algorithm.
    
    :math:`x_{n+1} = x_n - \\frac{f(x_n)}{f'(x_n)}`, where :math:`f(x)=x^2-a`.
    
    Arguments:
        x: a floating point number
    Returns:
        the square root of `x`
    Raises:
        ValueError: if `x` is not a number
    """
    if x < 0:
        raise ValueError("x must be positive")
    a = 1
    while abs(a**2 - x) > tol:
        a = .5 * (a + x / a)
    
    return a

assert abs(_sqrt(15) - math.sqrt(15)) < 1e-5

# also try this: assert abs(_sqrt(3) - math.sqrt(1)) < 1e-5

It is always nice to give a bit of context, so the first (and generally only) argument of an `Exception` is an error message.

So, what happen when you use a negative number?

In [ ]:
print(_sqrt(-2))

The exception was raised, interupting the process. In fact, the square root is never computed and nothing is printed!

You generally get a *stacktrace*, which is relatively useful in order to debug.

But it is sometimes useful to *catch* the error and act accordingly. To do so, put the call in a `try`/`except` bloc:

In [ ]:
try:
    _sqrt(-2)
except ValueError as e:
    print('Caught this:', e)

Notice that you can *catch* the error. This is useful to nicely report the error to the user. But you can do other things:

In [ ]:
try:
    _sqrt(-3)
except ValueError as e:
    print('This square root is imaginary, and its value is', _sqrt(3), 'i')

Note that you *catch* only the error indicated after `except`. For example,

In [ ]:
try:
    _sqrt('test')
except ValueError as e:
    print('caughth this:', e)

The `TypeError` was not *caught* and continue its way up to the main process. You can catch different type of errors by using the following construction:

In [ ]:
try:
    _sqrt('test')
except (ValueError, TypeError) as e:
    print('caught this:', e)

As you can see, the python objects also uses exceptions, which means that there is a bunch of already defined exceptions (postfixed by `Error`) which are available (see a list at https://docs.python.org/3/library/exceptions.html#concrete-exceptions). You can choose any of them for your own functions.

## Custom exceptions

You can define your own Exceptions, by deriving from `Exception`:

In [ ]:
class CustomException(Exception):
    pass

class AnotherCustomException(CustomException):
    def __init__(self, value: int, problem: str):
        super().__init__('Error with {}: {}'.format(value, problem))

raise AnotherCustomException(10, 'custom error')

Note that the inheritance applies to `except`, so:

In [ ]:
try:
    raise AnotherCustomException(10, 'custom error')
except CustomException as e:  # this catches AnotherCustomException as well!
    print('caught this:', e)

Finally, note that an error that is not *caught* crawls up in the stack trace. For example,

In [ ]:
def a():
    _sqrt(-2)

def b():
    try:
        a()
    except AnotherCustomException as e:
        print('error reported in b()', e)

def c():
    try:
        b()
    except ValueError as e:
        print('Error reported in a():', e)

c()  # calls `b()` which itself calls `a()`

The error is *raised* in `a()` but only *caught* in `c()` since, despite a `try`/`except` block in `b()`, the Exception is not *caught* there.

## Python principle: "*easier to ask for forgiveness than permission*"

Python as a few maxims like this. This one is well explained in [this *stackoverflow* answer](https://stackoverflow.com/a/11360880):

In [ ]:
my_dict = {}

# EAFP:
try:
    x = my_dict["key"]
except KeyError:
    pass

# LBYL ("Look before you leap"):
if "key" in my_dict:
    x = my_dict["key"]
else:
    pass


The LBYL version has to search the key inside the dictionary **twice**... and might be considered slightly less readable by some people ;)